In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [4]:
import pandas as pd
df1 = pd.read_csv("data/obj-001.csv")
df1.head()

,Unnamed: 0,object,pos_x,pos_y,pos_z,nor_x,nor_y,nor_z,type
0,0,sphere,0.523766,-0.501079,0.498178,0.523766,-0.501079,0.498178,good
1,1,sphere,-0.497427,-0.496840,0.518149,-0.497427,-0.496840,0.518149,good
2,2,sphere,0.515666,0.508871,0.496696,0.515666,0.508871,0.496696,good
3,3,sphere,-0.489167,0.514015,0.497168,-0.489167,0.514015,0.497168,good
4,4,sphere,-0.473849,-0.486696,-0.476986,-0.473849,-0.486696,-0.476986,good


In [7]:
# concatenate all the csv files in the data folder
import os
import pandas as pd
data_folder = "data"
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]
df_list = [pd.read_csv(f) for f in all_files]
df = pd.concat(df_list, ignore_index=True, axis=0)
df.head()

,Unnamed: 0,object,pos_x,pos_y,pos_z,nor_x,nor_y,nor_z,type
0,0,sphere,0.000924,-0.477336,0.538263,0.000924,-0.477336,0.538263,bad
1,1,sphere,-0.554023,-0.566356,0.118865,-0.554023,-0.566356,0.118865,bad
2,2,sphere,0.171024,0.313707,0.569393,0.171024,0.313707,0.569393,bad
3,3,sphere,-0.727502,0.205684,0.559479,-0.727502,0.205684,0.559479,bad
4,4,sphere,-1.049161,-0.779391,-0.983292,-1.049161,-0.779391,-0.983292,bad


In [8]:
df.describe()

,Unnamed: 0,pos_x,pos_y,pos_z,nor_x,nor_y,nor_z
count,4508.000000,4508.000000,4508.000000,4508.000000,4508.000000,4508.000000,4508.000000
mean,48.500000,0.013832,0.007780,0.013495,0.013832,0.007780,0.013495
std,28.291828,0.445538,0.451899,0.461386,0.445538,0.451899,0.461386
min,0.000000,-1.272178,-1.190460,-1.301119,-1.272178,-1.190460,-1.301119
25%,24.000000,-0.348681,-0.390462,-0.326444,-0.348681,-0.390462,-0.326444
50%,48.500000,0.002154,-0.020821,0.047727,0.002154,-0.020821,0.047727
75%,73.000000,0.373550,0.399634,0.418700,0.373550,0.399634,0.418700
max,97.000000,1.230102,1.242384,1.120505,1.230102,1.242384,1.120505


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4508 entries, 0 to 4507
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4508 non-null   int64  
 1   object      4508 non-null   object 
 2   pos_x       4508 non-null   float64
 3   pos_y       4508 non-null   float64
 4   pos_z       4508 non-null   float64
 5   nor_x       4508 non-null   float64
 6   nor_y       4508 non-null   float64
 7   nor_z       4508 non-null   float64
 8   type        4508 non-null   object 
dtypes: float64(6), int64(1), object(2)
memory usage: 317.1+ KB


In [10]:
df.drop(['Unnamed: 0', 'object'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4508 entries, 0 to 4507
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pos_x   4508 non-null   float64
 1   pos_y   4508 non-null   float64
 2   pos_z   4508 non-null   float64
 3   nor_x   4508 non-null   float64
 4   nor_y   4508 non-null   float64
 5   nor_z   4508 non-null   float64
 6   type    4508 non-null   object 
dtypes: float64(6), object(1)
memory usage: 246.7+ KB


In [12]:
df['type'].value_counts()

type
bad     2646
good    1862
Name: count, dtype: int64

In [13]:
df['type'].replace({'good': 1, 'bad': 0}, inplace=True)
df.head()

/tmp/ipykernel_594970/4095063675.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['type'].replace({'good': 1, 'bad': 0}, inplace=True)


,pos_x,pos_y,pos_z,nor_x,nor_y,nor_z,type
0,0.000924,-0.477336,0.538263,0.000924,-0.477336,0.538263,0
1,-0.554023,-0.566356,0.118865,-0.554023,-0.566356,0.118865,0
2,0.171024,0.313707,0.569393,0.171024,0.313707,0.569393,0
3,-0.727502,0.205684,0.559479,-0.727502,0.205684,0.559479,0
4,-1.049161,-0.779391,-0.983292,-1.049161,-0.779391,-0.983292,0


In [14]:
df = df.sample(frac=1).reset_index(drop=True)  # shuffle the dataframe

In [15]:
df.tail()

,pos_x,pos_y,pos_z,nor_x,nor_y,nor_z,type
4503,0.484496,-0.071760,-0.049902,0.484496,-0.071760,-0.049902,1
4504,0.732925,0.308060,0.362987,0.732925,0.308060,0.362987,0
4505,-0.265120,-0.328597,0.646148,-0.265120,-0.328597,0.646148,0
4506,-1.095571,0.113219,-0.911791,-1.095571,0.113219,-0.911791,0
4507,-0.623127,0.196424,0.084650,-0.623127,0.196424,0.084650,0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('type', axis=1), df['type'], test_size=0.2, random_state=42)

In [31]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x
    
###

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

input_size = X_train_tensor.shape[1]
model = BinaryClassifier(input_size)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
with torch.no_grad():
    predicted = model(X_test_tensor)
    predicted_classes = (predicted >= 0.5).float()
    acc = accuracy_score(y_test_tensor.squeeze().numpy(), predicted_classes.squeeze().numpy())
    print(f'Accuracy: {acc:.4f}')

Epoch [10/1000], Loss: 0.6615
Epoch [20/1000], Loss: 0.6293
Epoch [30/1000], Loss: 0.5845
Epoch [40/1000], Loss: 0.5679
Epoch [50/1000], Loss: 0.5458
Epoch [60/1000], Loss: 0.5196
Epoch [70/1000], Loss: 0.4961
Epoch [80/1000], Loss: 0.4751
Epoch [90/1000], Loss: 0.4598
Epoch [100/1000], Loss: 0.4449
Epoch [110/1000], Loss: 0.4287
Epoch [120/1000], Loss: 0.4147
Epoch [130/1000], Loss: 0.3995
Epoch [140/1000], Loss: 0.4000
Epoch [150/1000], Loss: 0.3946
Epoch [160/1000], Loss: 0.3747
Epoch [170/1000], Loss: 0.3576
Epoch [180/1000], Loss: 0.4561
Epoch [190/1000], Loss: 0.3446
Epoch [200/1000], Loss: 0.3395
Epoch [210/1000], Loss: 0.3266
Epoch [220/1000], Loss: 0.3184
Epoch [230/1000], Loss: 0.3162
Epoch [240/1000], Loss: 0.3242
Epoch [250/1000], Loss: 0.3190
Epoch [260/1000], Loss: 0.2926
Epoch [270/1000], Loss: 0.2837
Epoch [280/1000], Loss: 0.2749
Epoch [290/1000], Loss: 0.3037
Epoch [300/1000], Loss: 0.2636
Epoch [310/1000], Loss: 0.2576
Epoch [320/1000], Loss: 0.2585
Epoch [330/1000],

In [36]:
df_inference = pd.read_csv("test-data/test-obj-011.csv")
df_inference = df_inference.drop(['Unnamed: 0', 'object', 'type'], axis=1)

X_inference_tensor = torch.tensor(df_inference.values, dtype=torch.float32)

with torch.no_grad():
    inference_outputs = model(X_inference_tensor)
    inference_predicted_classes = (inference_outputs >= 0.5).float()
    print(inference_predicted_classes.squeeze().numpy())

[1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 0. 1.]


In [38]:
from collections import Counter
counter = Counter(inference_predicted_classes.squeeze().numpy())
most_frequent_labels, count = counter.most_common(1)[0]
print(f'Most frequent label: {most_frequent_labels} with count: {count}')

Most frequent label: 1.0 with count: 51
